###### imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib import transforms
import scipy
import scipy.stats as st
from scipy.integrate import odeint
import networkx as nx
import random
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from SequenceGenerator import MultiSequenceGenerator
from PoissonParetoSimulator import PoissonParetoBurstProcessSimulator

# Bianconi-Barabasi with pareto-poisson-burst as fitness function

### implementation

In [ ]:
class BianconiBarabashiPP():
    def draw_graph_for_time_slot(self, step_number):
        print(f'step number: {step_number}')
        fig = plt.figure(figsize=(50, 50))
        degrees = np.array([self.graph.degree(n) for n in self.graph.nodes()])
        node_size = degrees*100
        
        pos=nx.spring_layout(self.graph)
        cmap=plt.cm.viridis
        
        nodes = nx.draw_networkx_nodes(self.graph, pos, node_size=node_size, node_color=node_size, cmap=cmap)
        edges = nx.draw_networkx_edges(self.graph, pos)
        
        plt.colorbar(nodes)
        plt.axis('off')
        plt.title(f'step number: {step_number}')
        plt.show()

    
    
    def append_nodes(self):
        time_intervals = int(self.step_number/self.time_slot_number)
        time_slots = list(range(time_intervals, self.step_number+1, time_intervals))
        
        for new_node in range(self.initial_node_number+1, self.step_number+self.initial_node_number+1):
            degrees = [val for (node, val) in sorted(self.graph.degree(), key=lambda pair: pair[0])]
            summation = np.sum([self.fitness[i]*degrees[i] for i in range(len(degrees))])
            probabilities = [(self.fitness[i]*degrees[i])/summation for i in range(len(degrees))]
            probabilities_cum = np.cumsum(probabilities)
            interval_max = probabilities_cum[len(probabilities_cum) - 1]
            
            self.graph.add_node(new_node)
            
            for _ in range(self.new_node_link):
                random_number = np.random.uniform(0.0, interval_max)
                connected_node = sum(i < random_number for i in probabilities_cum) + 1 # pluse one is because node numbers starts from 1 not zero
                self.graph.add_edge(new_node, connected_node)

            step_number = new_node - (self.initial_node_number+1)
            if (step_number in time_slots):
                self.draw_graph_for_time_slot(step_number)
    
    def __init__(self, initial_node_number=3, step_number=100, new_node_link=3, fitness=[], time_slot_number=10):
        self.initial_node_number=initial_node_number
        self.step_number=step_number
        self.new_node_link=new_node_link
        self.fitness = fitness
        self.time_slot_number = time_slot_number
        self.graph = nx.complete_graph(self.initial_node_number+1)
        self.graph.remove_node(0)
        self.append_nodes()
        self.set_degree_distribution()

    def set_degree_distribution(self):
        self.degree_distribution = np.array([self.graph.degree(n) for n in self.graph.nodes()])

    def draw_degree_distribution(self):
        sns.displot(self.degree_distribution, kde=True)
        plt.title('degree distribution')
        plt.xlabel('degree')
        plt.ylabel('occurance of each degree')
        plt.show()

        sns.displot(self.degree_distribution, kde=True)
        plt.title('degree distribution semilog')
        plt.xlabel('degree')
        plt.ylabel('occurance of each degree')
        plt.semilogy()
        plt.show()

        sns.displot(self.degree_distribution, kde=True)
        plt.title('degree distribution log-log')
        plt.xlabel('degree')
        plt.ylabel('occurance of each degree')
        plt.xscale('log')
        plt.yscale('log')
        plt.show()

    def draw_graph(self):
        fig = plt.figure(figsize=(50, 50))
        node_size = self.degree_distribution*100
        
        pos=nx.spring_layout(self.graph)
        cmap=plt.cm.viridis
        
        nodes = nx.draw_networkx_nodes(self.graph, pos, node_size=node_size, node_color=node_size, cmap=cmap)
        edges = nx.draw_networkx_edges(self.graph, pos)
        
        plt.colorbar(nodes)
        plt.axis('off')
        plt.show()

### simulation

#### paretian-poisson burst process

##### parameters

###### Poisson-Pareto burst process

In [ ]:
total_time = 333
lam_list=[5.0]
hurst_list=[0.75]
burst_duration_mean = 4.0
has_pre_burst = True

###### bianconi-barabasi

In [ ]:
size = total_time*3
initial_node_number = 3
step_number=size-initial_node_number
new_node_link=3

##### generating values

###### poisson pareto process

In [ ]:
ppbp_sim = PoissonParetoBurstProcessSimulator(total_time=total_time, lam_list=lam_list, hurst_list=hurst_list, burst_duration_mean=burst_duration_mean, has_pre_burst=has_pre_burst, least_interval_length=None)
sim_res = ppbp_sim.simulate()

In [ ]:
fitness = sim_res[0][0]
sim_res[0][0].shape

###### bianconi-barabasi

In [ ]:
bbpp = BianconiBarabashiPP(initial_node_number=initial_node_number, step_number=step_number, new_node_link=new_node_link, fitness=fitness)

##### plotting results

###### degree distribution

In [ ]:
bbpp.draw_degree_distribution()

###### graph

In [ ]:
bbpp.draw_graph()

#### paretian-poisson process from unified model

##### parameters

###### paretian-poisson process

In [ ]:
def power_law_trajectory_explosion_time(initial_value, specific_initial_value, epsiolon): # F(u)
    return initial_value / np.power(np.power(specific_initial_value/initial_value, -epsilon)-1, 1.0/epsilon)

largest_initial_val = 0.001 # v in paper
size = 1000
epsilon = 1.0
c = 1
start_interval = 0.99

###### bianconi-barabasi

In [ ]:
initial_node_number = 3
step_number=size-initial_node_number
new_node_link=3

##### generatin values

###### paretian-poisson process

In [ ]:
initial_value_list = np.random.uniform(start_interval, largest_initial_val, size) * largest_initial_val

result = [power_law_trajectory_explosion_time(largest_initial_val, initial_value, epsilon) for initial_value in initial_value_list]

In [ ]:
fitness = result

###### bianconi-barabasi

In [ ]:
bbpp = BianconiBarabashiPP(initial_node_number=initial_node_number, step_number=step_number, new_node_link=new_node_link, fitness=fitness)

##### drawing plots

###### degree distribution

In [ ]:
bbpp.draw_degree_distribution()

###### graph

In [ ]:
bbpp.draw_graph()

#### paretian-poisson process our model

##### parameters

###### pareto-poisson

In [ ]:
def equation3(U, v, epsilon):
    return v / np.power(np.power(U, -epsilon) - 1.0 , 1.0/epsilon)

def equation5(data, c, epsilon):
    return c * epsilon / np.power(data, 1.0 + epsilon)

def generate_our_pareto_poisson(v, epsilon, c, size, should_sort=False):
    U = np.random.uniform(0.0, 1.0, size-1) * v
    U = np.append(U, v)

    if should_sort:
        U = np.sort(U)
    
    X = equation3(U, v, epsilon)
    return equation5(X, c, epsilon)

v = .9# v in the paper
size = 1000
epsilon = 1.
c = 1.

final_min_value = 1
final_max_value = 40

###### bianconi-barabasi

In [ ]:
initial_node_number = 3
step_number=size-initial_node_number
new_node_link=3

##### generating values

###### pareto-poisson

In [ ]:
result = generate_our_pareto_poisson(v, epsilon, c, size, should_sort=True)

In [ ]:
fitness = result

##### bianconi-barabasi

In [ ]:
bbpp = BianconiBarabashiPP(initial_node_number=initial_node_number, step_number=step_number, new_node_link=new_node_link, fitness=fitness)

##### draw plots

###### degree distribution

In [ ]:
bbpp.draw_degree_distribution()

###### graph

In [ ]:
bbpp.draw_graph()

# Bianconi-Barabasi with pareto-poisson-burst as probability

### implementation

In [ ]:
class BianconiBarabashiPP_Probability():
    def draw_graph_for_time_slot(self, step_number):
        print(f'step number: {step_number}')
        fig = plt.figure(figsize=(50, 50))
        degrees = np.array([self.graph.degree(n) for n in self.graph.nodes()])
        node_size = degrees*100
        
        pos=nx.spring_layout(self.graph)
        cmap=plt.cm.viridis
        
        nodes = nx.draw_networkx_nodes(self.graph, pos, node_size=node_size, node_color=node_size, cmap=cmap)
        edges = nx.draw_networkx_edges(self.graph, pos)
        
        plt.colorbar(nodes)
        plt.axis('off')
        plt.title(f'step number: {step_number}')
        plt.show()

    
    
    def append_nodes(self):
        time_intervals = int(self.step_number/self.time_slot_number)
        time_slots = list(range(time_intervals, self.step_number+1, time_intervals))
        
        for new_node in range(self.initial_node_number+1, self.step_number+self.initial_node_number+1):
            probabilities_cum = self.probabilities_cum[:self.graph.number_of_nodes()]
            interval_max = probabilities_cum[len(probabilities_cum) - 1]
            
            self.graph.add_node(new_node)
            
            for _ in range(self.new_node_link):
                random_number = np.random.uniform(0.0, interval_max)
                connected_node = sum(i < random_number for i in probabilities_cum) + 1 # pluse one is because node numbers starts from 1 not zero
                self.graph.add_edge(new_node, connected_node)

            step_number = new_node - (self.initial_node_number+1)
            if (step_number in time_slots):
                self.draw_graph_for_time_slot(step_number)
    
    def __init__(self, initial_node_number=3, step_number=100, new_node_link=3, probabilities=[], time_slot_number=10):
        self.initial_node_number=initial_node_number
        self.step_number=step_number
        self.new_node_link=new_node_link
        self.probabilities = probabilities
        self.probabilities_cum = np.cumsum(self.probabilities)
        self.time_slot_number = time_slot_number
        self.graph = nx.complete_graph(self.initial_node_number+1)
        self.graph.remove_node(0)
        self.append_nodes()
        self.set_degree_distribution()

    def set_degree_distribution(self):
        self.degree_distribution = np.array([self.graph.degree(n) for n in self.graph.nodes()])

    def draw_degree_distribution(self):
        sns.displot(self.degree_distribution, kde=True)
        plt.title('degree distribution')
        plt.xlabel('degree')
        plt.ylabel('occurance of each degree')
        plt.show()

        sns.displot(self.degree_distribution, kde=True)
        plt.title('degree distribution semilog')
        plt.xlabel('degree')
        plt.ylabel('occurance of each degree')
        plt.semilogy()
        plt.show()

        sns.displot(self.degree_distribution, kde=True)
        plt.title('degree distribution log-log')
        plt.xlabel('degree')
        plt.ylabel('occurance of each degree')
        plt.xscale('log')
        plt.yscale('log')
        plt.show()

    def draw_graph(self):
        fig = plt.figure(figsize=(50, 50))
        node_size = self.degree_distribution*100
        
        pos=nx.spring_layout(self.graph)
        cmap=plt.cm.viridis
        
        nodes = nx.draw_networkx_nodes(self.graph, pos, node_size=node_size, node_color=node_size, cmap=cmap)
        edges = nx.draw_networkx_edges(self.graph, pos)
        
        plt.colorbar(nodes)
        plt.axis('off')
        plt.show()

### simulation

#### paretian-poisson burst process

##### parameters

###### paretian-poisson

In [ ]:
total_time = 333
lam_list=[5.0]
hurst_list=[0.75]
burst_duration_mean = 4.0
has_pre_burst = True

###### bianconi-barabasi

In [ ]:
size = total_time*3
initial_node_number = 3
step_number=size-initial_node_number
new_node_link=3

##### generating values

###### poisson pareto process

In [ ]:
ppbp_sim = PoissonParetoBurstProcessSimulator(total_time=total_time, lam_list=lam_list, hurst_list=hurst_list, burst_duration_mean=burst_duration_mean, has_pre_burst=has_pre_burst, least_interval_length=None)
sim_res = ppbp_sim.simulate()

In [ ]:
probabilities = sim_res[0][0]

###### bianconi-barabasi

In [ ]:
bbpp_prob = BianconiBarabashiPP_Probability(initial_node_number=initial_node_number, step_number=step_number, new_node_link=new_node_link, probabilities=probabilities)

##### drawing results

###### degree distribution

In [ ]:
bbpp_prob.draw_degree_distribution()

###### graph

In [ ]:
bbpp_prob.draw_graph()

#### paretian-poisson process from unified model

##### parameters

###### paretian-poisson process

In [ ]:
def power_law_trajectory_explosion_time(initial_value, specific_initial_value, epsiolon): # F(u)
    return initial_value / np.power(np.power(specific_initial_value/initial_value, -epsilon)-1, 1.0/epsilon)

largest_initial_val = 0.001 # v in paper
size = 1000
epsilon = 1.0
c = 1
start_interval = 0.99

###### bianconi-barabasi

In [ ]:
initial_node_number = 3
step_number=size-initial_node_number
new_node_link=3

##### generatin values

###### paretian-poisson process

In [ ]:
initial_value_list = np.random.uniform(start_interval, largest_initial_val, size) * largest_initial_val

result = [power_law_trajectory_explosion_time(largest_initial_val, initial_value, epsilon) for initial_value in initial_value_list]

In [ ]:
probabilities = result

###### bianconi-barabasi

In [ ]:
bbpp_prob = BianconiBarabashiPP_Probability(initial_node_number=initial_node_number, step_number=step_number, new_node_link=new_node_link, probabilities=probabilities)

##### drawing results

###### degree distribution

In [ ]:
bbpp_prob.draw_degree_distribution()

###### graph

In [ ]:
bbpp_prob.draw_graph()

#### paretian-poisson process our model

##### parameters

###### pareto-poisson

In [ ]:
def equation3(U, v, epsilon):
    return v / np.power(np.power(U, -epsilon) - 1.0 , 1.0/epsilon)

def equation5(data, c, epsilon):
    return c * epsilon / np.power(data, 1.0 + epsilon)

def generate_our_pareto_poisson(v, epsilon, c, size, should_sort=False):
    U = np.random.uniform(0.0, 1.0, size-1) * v
    U = np.append(U, v)

    if should_sort:
        U = np.sort(U)
    
    X = equation3(U, v, epsilon)
    return equation5(X, c, epsilon)

v = .9# v in the paper
size = 1000
epsilon = 1.
c = 1.

final_min_value = 1
final_max_value = 40

###### bianconi-barabasi

In [ ]:
initial_node_number = 3
step_number=size-initial_node_number
new_node_link=3

##### generating values

###### pareto-poisson

In [ ]:
result = generate_our_pareto_poisson(v, epsilon, c, size, should_sort=True)

In [ ]:
probabilities = result

###### bianconi-barabasi

In [ ]:
bbpp_prob = BianconiBarabashiPP_Probability(initial_node_number=initial_node_number, step_number=step_number, new_node_link=new_node_link, probabilities=probabilities)

##### drawing plot

###### degree distribution

In [ ]:
bbpp_prob.draw_degree_distribution()

###### graph

In [ ]:
bbpp_prob.draw_graph()